# Анализ защищенности систем искусственного интеллекта
# Практическая работа № 6
# Атака по переносу (Transfer Attack) на модели ИИ
## Выполнил студент группы ББМО-02-23
## Макаров Михаил Максимович


# Шаг 1: Загрузка и создание двух различных моделей

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
# Загрузка данных MNIST
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
# Нормализация данных
train_images = train_images / 255.0
test_images = test_images / 255.0
# Преобразование меток в one-hot encoding
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)
# Модель 1: Простая полносвязная нейронная сеть
model1 = Sequential([
  Flatten(input_shape=(28, 28)),
  Dense(128, activation='relu'),
  Dense(10, activation='softmax')])
# Компиляция модели
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics= ['accuracy'])
# Обучение модели
model1.fit(train_images, train_labels, epochs=5)
# Сохранение модели
model1.save('mnist_model1.h5')
# Модель 2: Свёрточная нейронная сеть (CNN)
model2 = Sequential([
  Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
  MaxPooling2D((2, 2)),
  Flatten(),
  Dense(128, activation='relu'),
  Dense(10, activation='softmax')
])
# Компиляция модели
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics= ['accuracy'])
# Обучение модели
model2.fit(train_images.reshape(-1, 28, 28, 1), train_labels, epochs=5)
# Сохранение модели
model2.save('mnist_model2.h5')

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.8806 - loss: 0.4295
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9622 - loss: 0.1281
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9770 - loss: 0.0784
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9829 - loss: 0.0570
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.9875 - loss: 0.0417


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 43s 22ms/step - accuracy: 0.9053 - loss: 0.3181
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 22ms/step - accuracy: 0.9841 - loss: 0.0536
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 80s 21ms/step - accuracy: 0.9905 - loss: 0.0306
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 42s 22ms/step - accuracy: 0.9936 - loss: 0.0216
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 42s 22ms/step - accuracy: 0.9951 - loss: 0.0144


# Шаг 2: Теория атаки по переносу

Атака по переносу заключается в использовании противоречивых примеров, сгенерированных для одной модели, для атаки на другую модель. Такая возможность возникает из-за того, что модели, даже если они основаны на разных архитектурах, могут обладать схожими уязвимостями и допускать схожие ошибки при обработке определённых данных.

# Шаг 3: Реализация атаки FGSM на первую модель


In [2]:
import numpy as np
# Функция для реализации FGSM атаки
def fgsm_attack(image, epsilon, gradient):
  # Применение знака градиента к изображению
  perturbed_image = image + epsilon * np.sign(gradient)
  # Обрезка значений, чтобы они оставались в пределах [0,1]
  perturbed_image = np.clip(perturbed_image, 0, 1)
  return perturbed_image
# Вычисление градиента
def generate_adversarial_example(model, image, label, epsilon):
    # Превращаем изображение в формат, подходящий для модели
    image = tf.convert_to_tensor(image.reshape((1, 28, 28, 1)))

    # Если label — это one-hot вектор, преобразуем его в индекс
    if len(label.shape) > 1 and label.shape[1] > 1:
        label = np.argmax(label)
    label = tf.convert_to_tensor(label)

    with tf.GradientTape() as tape:
        tape.watch(image)
        prediction = model(image)
        loss = tf.keras.losses.categorical_crossentropy(label[None], prediction)

    gradient = tape.gradient(loss, image)

    # Применяем FGSM
    adversarial_image = fgsm_attack(image.numpy(), epsilon, gradient.numpy())

    # Убедимся, что adversarial_image имеет правильную форму
    return np.reshape(adversarial_image, (28, 28, 1))

def generate_adversarial_dataset(model, images, labels, epsilon):
    adversarial_images = []
    for i in range(len(images)):
        adv_image = generate_adversarial_example(model, images[i], labels[i], epsilon)
        adversarial_images.append(adv_image.reshape(28, 28))

    adversarial_images = np.array(adversarial_images)

    # Проверка формы
    print("Shape of adversarial_images:", adversarial_images.shape)

    return adversarial_images


# Генерация противоречивых примеров для первой модели
epsilon = 0.1
adversarial_images_model1 = generate_adversarial_dataset(model1, test_images, test_labels, epsilon)

Shape of adversarial_images: (10000, 28, 28)


# Шаг 4: Оценка противоречивых примеров на обеих моделях

In [3]:
# Оценка первой модели на противоречивых примерах
loss1, acc1 = model1.evaluate(adversarial_images_model1, test_labels)
print(f'Accuracy of model1 on adversarial examples: {acc1}')

# Оценка второй модели на противоречивых примерах (перенос атаки)
adversarial_images_model1_reshaped = adversarial_images_model1.reshape(-1, 28, 28, 1)
loss2, acc2 = model2.evaluate(adversarial_images_model1_reshaped, test_labels)
print(f'Accuracy of model2 on adversarial examples from model1: {acc2}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0724 - loss: 7.1211
Accuracy of model1 on adversarial examples: 0.094200000166893
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9594 - loss: 0.1374
Accuracy of model2 on adversarial examples from model1: 0.9664999842643738


# Шаг 5: Анализ переносимости атак

In [4]:
# Генерация противоречивых примеров для второй модели
adversarial_images_model2 = generate_adversarial_dataset(model2,
test_images.reshape(-1, 28, 28, 1), test_labels, epsilon)

# Оценка первой модели на противоречивых примерах второй модели
loss3, acc3 = model1.evaluate(adversarial_images_model2.reshape(-1, 28,
28), test_labels)
print(f'Accuracy of model1 on adversarial examples from model2: {acc3}')

Shape of adversarial_images: (10000, 28, 28)
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8990 - loss: 0.3265
Accuracy of model1 on adversarial examples from model2: 0.9193999767303467


# Вывод:

В ходе исследования было изучено влияние атаки по переносу, при которой противоречивые примеры, созданные для одной модели, применялись для атаки на другую. Для генерации таких примеров использовался метод FGSM (Fast Gradient Sign Method).

Результаты эксперимента показали следующее:

Модель 1, для которой были сгенерированы противоречивые примеры с помощью FGSM, испытала огромное снижение точности.

Модель 2, напротив, оказалась более устойчивой к атаке

Кроме того, были созданы противоречивые примеры с использованием FGSM для модели 2 и протестированы на модели 1. В этом случае точность модели 1 также снизилась, но падение составило всего около 3%.

Таким образом, результаты исследования подтверждают, что атака по переносу с использованием FGSM может значительно снизить точность модели, особенно если примеры генерируются непосредственно для неё. В то же время, модель с более высокой устойчивостью демонстрирует меньшую восприимчивость к подобным атакам.

